In [ ]:
import matplotlib as plt
import pandas as pd
from pandas import DatetimeIndex
pd.options.display.max_colwidth = 100
import pickle
import datetime as dt

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import time

from bs4 import BeautifulSoup as BS

import requests
from urllib.request import urlopen

import tensorflow as tf
import itertools

In [ ]:
#Create a list of the pages we wanna search
pages = list(range(2,11)) #2-10 - page one has uniuqe url

#Create a list of with first and last date of each month to not max the the search results
#how far back do we want to go
year_range = range(2020,2022) #2020-2021
month_range = list(range(1,13)) #all 12 months
dates_range = list(itertools.product(year_range, month_range)) #combine the two in all posiblilties 

#Find the first date of the month
dates_start = [d.strftime('%Y-%m-%d') for d in pd.date_range(start="1/1/2020", end="2021/12/1", freq='MS')]

#Find the last date of the month
dates_end = []
for year, month in dates_range:
    date_end = pd.date_range('{}-{}'.format(year, month), periods=1, freq='M').strftime('%Y-%m-%d')
    dates_end.append(date_end)
    
#Create a df with the first and last dates
dates_start_df = pd.DataFrame(dates_start)
dates_end_df = pd.DataFrame(dates_end)
dates_df = pd.concat([dates_start_df, dates_end_df], axis=1)
dates_df.columns = ['Start','End']
dates_list = dates_df.values.tolist()

#make them in to two list 
sm_list_start = dates_df['Start'].values.tolist()
sm_list_end = dates_df['End'].values.tolist()


#create a list with page range and date posiblities
tester = list(itertools.product(pages, dates_list))

#make the new list with all the posibilites in three sperat lists
tester = pd.DataFrame(tester)
tester.columns = ['Pages','Dates']
tester[['Start','End']] = pd.DataFrame(tester['Dates'].tolist(), index=tester.index)
tester = tester.drop(columns='Dates')
list_pages = tester['Pages'].values.tolist()
list_start = tester['Start'].values.tolist()
list_end = tester['End'].values.tolist()


In [ ]:
#date posiblites
print(len(sm_list_start))
print(len(sm_list_end))

#all posiblities
print(len(list_pages))
print(len(list_start))
print(len(list_end))

24
24
216
216
216


In [ ]:
timeout = time.time() + 60*50   # 5 minutes from now
url="https://www.ft.com/search?q=pfizer&dateFrom=2020-01-01&sort=date&expandRefinements=true"

options = Options()
options.add_argument("--window-size=2560,1440")
options.add_argument("disable-notifications")
driver = webdriver.Chrome("C:\python_file\chromedriver.exe", options=options)

driver.get(url)
driver.set_window_position(-2560, 0)
driver.maximize_window()

stories_list=[]
for i, j, k in zip(list_pages, list_start, list_end):
    time.sleep(5)
    test = 0
    driver.get(f'https://www.ft.com/search?q=pfizer&page={i}&dateTo={j}&dateFrom={k}&sort=date&expandRefinements=true')
    
    page_source = driver.page_source
    soup = BS(page_source, 'html.parser')
        
    stories = soup.find_all('div', {'class':'o-teaser__content'})
    for story in stories:
        headline = story.find('a', {'class':"js-teaser-heading-link"})
        date = story.find('time', {'class':"o-teaser__timestamp-date"})
        #des = story.find('p', {'class':"o-teaser__standfirst"}).find('span')
        des = story.find('span')
        if date:
        # Build a dict that HAS a summary
            story_dict = {
                'Headline': headline.text,
                'Date': date['datetime'],
                'Description': des
            }
        else:
                story_dict = {
                    'Headline': headline.text,
                    'Description': des
            }
               
        if story_dict not in stories_list:
            # Add the dict to our list
            stories_list.append(story_dict) 
            
    if test == 5 or time.time() > timeout:
        break
df = pd.DataFrame(stories_list)

C:\Users\NiXxXO\AppData\Local\Temp/ipykernel_14924/1701963193.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\python_file\chromedriver.exe", options=options)


In [ ]:
df

,Headline,Date,Description
0,Scientists raise questions over lasting immunity from Covid-19 vaccine,2020-07-16T04:01:21+0000,"[...seemed to induce a better response than the vaccine being developed by , [Pfizer], with Bio..."
1,Markets Now - Wednesday 15th July 2020,2020-07-15T10:41:19+0000,"[...US] election data is possible, but our base case is mid-November: , [Pfizer], has stated th..."
2,Moderna’s Covid-19 vaccine produces antibodies in early trial,2020-07-14T21:01:48+0000,"[...to teach the immune system to detect the invader. Other companies including , [Pfizer], and..."
3,"Facebook has caused ‘serious setbacks’ for civil rights, report finds",2020-07-08T15:18:02+0000,"[...on its platform, with about 1,000 brands — including Unilever, Ford and , [Pfizer], — agree..."
4,Spain’s BBVA breaks ground by issuing risky ‘green’ debt,2020-07-08T03:00:48+0000,"[...slapping the label on investments that may not withstand scrutiny.US drugmaker , [Pfizer], ..."
...,...,...,...
1380,AstraZeneca still suffering growing pains,2020-11-05T16:56:22+0000,[Lombard]
1381,"Covid-19 vaccine market worth $10bn a year, analysts say",2020-11-05T05:00:32+0000,"[...to discuss the potential of the Covid-19 vaccine market. Merck and , [Pfizer], were not ava..."
1382,AstraZeneca misses target for delivery of UK Covid vaccine,2020-11-04T17:18:02+0000,"[...AstraZeneca vaccine will be supplemented by up to 10m doses of the , [Pfizer], /BioNTech vac..."
1383,FT Health: The dangers of data,2020-11-04T14:21:50+0000,"[...J&J got the go-ahead to resume vaccine trials, but the , [Pfizer], chief called for patienc..."


In [ ]:
timeout = time.time() + 60*50   # 5 minutes from now
url="https://www.ft.com/search?q=pfizer&dateFrom=2020-01-01&sort=date&expandRefinements=true"

options = Options()
options.add_argument("--window-size=2560,1440")
options.add_argument("disable-notifications")
driver = webdriver.Chrome("C:\python_file\chromedriver.exe", options=options)

driver.get(url)
driver.set_window_position(-2560, 0)
driver.maximize_window()

stories_list1=[]
for j, k in zip(sm_list_start, sm_list_end):
    test = 0
    time.sleep(5)
    driver.get(f'https://www.ft.com/search?q=pfizer&dateTo={j}&dateFrom={k}&sort=date&expandRefinements=true')
    
    page_source = driver.page_source
    soup = BS(page_source, 'html.parser')
        
    stories = soup.find_all('div', {'class':'o-teaser__content'})
    for story in stories:
        headline = story.find('a', {'class':"js-teaser-heading-link"})
        date = story.find('time', {'class':"o-teaser__timestamp-date"})
        #des = story.find('p', {'class':"o-teaser__standfirst"}).find('span')
        des = story.find('span')
        if date:
        # Build a dict that HAS a summary
            story_dict = {
                'Headline': headline.text,
                'Date': date['datetime'],
                'Description': des
            }
        else:
                story_dict = {
                    'Headline': headline.text,
                    'Description': des
            }
               
        if story_dict not in stories_list1:
            # Add the dict to our list
            stories_list1.append(story_dict) 
            
    if test == 5 or time.time() > timeout:
        break
df1 = pd.DataFrame(stories_list1)

C:\Users\NiXxXO\AppData\Local\Temp/ipykernel_14924/3349221578.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\python_file\chromedriver.exe", options=options)


In [ ]:
print(len(df))
print(len(df1))
df2 = pd.concat([df,df1], axis=0)
print(df2.isna().sum())

1385
499
Headline        0
Date           66
Description     0
dtype: int64


In [ ]:
df2['Date'] = pd.to_datetime(df2['Date']).dt.tz_localize(None)
df2 = df2.dropna(subset=['Date'])
df2 = df2.drop_duplicates(subset =['Headline', 'Date','Description'], keep='first')
df2 = df2.sort_values(by=['Date'])
df2.shape[0]

1818

In [ ]:
df2.to_csv(r'C:\python_file\save\ft.csv')

In [ ]:
df2

,Headline,Date,Description
8,Pfizer and other drugmakers start 2020 with price rises,2020-01-03 18:15:00,"[[Pfizer], and other drugmakers start 2020 with price rises]"
7,The hedge fund winners and losers of 2019,2020-01-10 05:00:34,"[ , [Premium], [ content]]"
6,Hedge fund veteran looks to cash in on biotech boom,2020-01-16 04:00:31,"[...as Array BioPharma, which was bought for $11.4bn last year by , [Pfizer], , have benefited d..."
5,"Movers and shakers, January 20",2020-01-20 03:00:39,"[...head of strategic platform investment solutions. Ian Read, former chief executive of , [Pfiz..."
4,"Markets not live, Wednesday 22nd January 2020",2020-01-22 12:24:42,"[...to prove dormant rather than dead, the 2014 Sunday Times scoop of , [Pfizer], 's on-off-on-o..."
...,...,...,...
478,"Two Pfizer doses give 70% protection against hospitalisation from Omicron, study shows",2021-12-14 13:26:55,"[Two , [Pfizer], doses give 70% protection against hospitalisation from Omicron, study shows]"
477,Pfizer says Covid drug offers strong protection and should work against Omicron,2021-12-14 13:53:43,"[[Pfizer], says Covid drug offers strong protection and should work against Omicron]"
476,Episode 51,2021-12-14 14:32:05,[3 min listen]
475,"How did the UK reach 200,000 Omicron infections in one day?",2021-12-14 19:50:14,"[...minute observation period after the administration of the mRNA Covid vaccines from , [Pfizer..."


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=520f7743-fdb5-41d4-ba65-deb67c6338e8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>